# Test of whether parameters were successfully applied
Provide the path to an ouput sqlite database and this code will check the attributes of results against the parameters that were supposed to be applied.

In [5]:
database = "T:/Occurrence_Records/GBIF_API_TEST.sqlite"

In [6]:
def validate_output(database):
    '''
    Compares the occurrence record attributes to the filters that were supposed to be applied.
    
    
    PARAMETERS
    ---------
    database: path to a wrangler output database.  Like ""Z:/Occurrence_Records/test1.sqlite""
    
    RESULTS
    -------
    prints messages if tests are failed. No output indicates all tests were passed.
    '''
    
    return

In [7]:
import sqlite3
import pandas as pd
import geopandas as gpd
import shapely

# Connect to a database
conn = sqlite3.connect(database)
cursor = conn.cursor()

# Get the taxon concept -------------------------------------------
taxon_concept = (pd.read_sql(sql="SELECT * FROM taxon_concept;", con=conn)
                 .rename({"index": "key", "0": "value"}, axis=1)
                 .set_index("key"))
print(taxon_concept)

# Get the filter set that was applied -------------------------------
filter_set = (pd.read_sql(sql="SELECT * FROM filter_set", con=conn)
              .rename({"index": "parameter", "0": "value"}, axis=1)
              .set_index("parameter"))
print(filter_set)

# Presence only -----------------------------------------------------
presabs = [x[0] for x in conn.execute("SELECT DISTINCT occurrenceStatus FROM occurrence_records;").fetchall()]
if presabs != ["PRESENT"] and presabs != ['nan', 'PRESENT']:
    print("!!! Failed occurrenceStatus test: " + str(presabs))

# DWCA archive ----------------------------------------------------
if filter_set.loc["get_dwca", "value"] == "True":
    try:
        conn.execute("SELECT download_key FROM GBIF_download_info;").fetchall()
    except:
        print("!!! Failed DWCA download key test")
        
# Latitude range --------------------------------------------------
lat_range = filter_set.loc["lat_range", "value"]
if lat_range != "None":
    lat_range = [float(x) for x in lat_range.split(",")]
    max_lat = float(conn.execute("SELECT MAX(decimalLatitude) FROM occurrence_records;").fetchall()[0][0])
    min_lat = float(conn.execute("SELECT MIN(decimalLatitude) FROM occurrence_records;").fetchall()[0][0])
    if (min_lat < lat_range[0] or max_lat > lat_range[1]):
        print("!!! Failed test of decimalLatitude values")

# Longitude range ------------------------------------------------
lon_range = filter_set.loc["lon_range", "value"]
if lon_range != "None":
    lon_range = [float(x) for x in lon_range.split(",")]
    max_lon = float(conn.execute("SELECT MAX(decimalLongitude) FROM occurrence_records;").fetchall()[0][0])
    min_lon = float(conn.execute("SELECT MIN(decimalLongitude) FROM occurrence_records;").fetchall()[0][0])
    if (min_lon < lon_range[0] or max_lon > lon_range[1]):
        print("!!! Failed test of decimalLongitude values")
        
# Years range ----------------------------------------------------
yrs_range = filter_set.loc["years_range", "value"]
if yrs_range != "None":
    yrs_range = [float(x) for x in yrs_range.split(",")]
    max_yr = float(conn.execute("SELECT MAX(strftime('%Y', eventDate)) FROM occurrence_records;").fetchall()[0][0])
    min_yr = float(conn.execute("SELECT MIN(strftime('%Y', eventDate)) FROM occurrence_records;").fetchall()[0][0])
    if (min_yr < yrs_range[0] or max_yr > yrs_range[1]):
        print("!!! Failed test of year (eventDate) values")
        
# Months range ---------------------------------------------------
months_range = filter_set.loc["months_range", "value"]
if months_range != "None":
    months_range = [float(x) for x in months_range.split(",")]
    max_month = float(conn.execute("SELECT MAX(strftime('%m', eventDate)) FROM occurrence_records;").fetchall()[0][0])
    min_month = float(conn.execute("SELECT MIN(strftime('%m', eventDate)) FROM occurrence_records;").fetchall()[0][0])
    # Months range could be like 1,12
    if months_range[0] < months_range[1]:
        if (min_month < months_range[0] or max_month > months_range[1]):
            print("!!! Failed test of month (eventDate) values")
    # Months range could be like 11,3
    if months_range[0] > months_range[1]:
        no_months = list(range(months_range[1] + 1, months_range[0]) -1)
        months = conn.execute("SELECT DISTINCT strftime('%m', eventDate) FROM occurrence_records;").fetchall()
        months = [int(x[0]) for x in months]
        if len(set(months) & set(no_months)) != 0:
            print("!!! Failed test of month (eventDate) values")
            
# eBird ID --------------------------------------------------------
ebd_yn = [x[0] for x in conn.execute("SELECT DISTINCT source FROM occurrence_records;").fetchall()]
ebirdid = conn.execute("SELECT DISTINCT ebird_id FROM occurrence_records;").fetchall()[0][0]
out_ebirdid = taxon_concept.loc["EBIRD_ID", "value"]
if 'eBird' in ebd_yn:
    if ebirdid != out_ebirdid:
        print("!!! Failed test of ebird_id")
        
# GBIF ID ---------------------------------------------------------
gbifid = conn.execute("SELECT DISTINCT gbif_id FROM occurrence_records;").fetchall()[0][0]
out_gbifid = taxon_concept.loc["GBIF_ID", "value"]
if gbifid != out_gbifid:
    print("!!! Failed test of gbif_id")
    
# Maximum coordinate uncertainty -----------------------------------
mcu_out = float(conn.execute("SELECT MAX(coordinateUncertaintyInMeters) FROM occurrence_records;").fetchall()[0][0])
dd = float(taxon_concept.loc["detection_distance_m", "value"])
mcu = float(filter_set.loc["max_coordinate_uncertainty", "value"])
if mcu_out > mcu:
    print("!!! Failed test of maximum coordinate uncertainty. Max = " + str(mcu_out))
    
# Various ------------------------------------------------------
multilists = {"issues_omit": "issues", "sampling_protocols_omit": "samplingProtocol", "bases_omit": "basisOfRecord", 
              "datasets_omit": "datasetName", "collection_codes_omit": "collectionCode", "institutions_omit": "institutionID"}

def test_multilist(attribute):
    '''Test attribute values that involove multilists'''
    values = set([])
    data = [x[0].split("; ") for x in conn.execute("SELECT DISTINCT {0} FROM occurrence_records;".format(multilists[attribute])).fetchall()]
    for x in data:
        values = set(x) | values
    invalid = set(filter_set.loc[attribute, "value"].replace("'", "").replace("[", "").replace("]", "").split(", "))
    if len(invalid & values) >=1:
        print("!!! Failed test of {0}.".format(multilists[attribute]))

for x in multilists.keys():
    test_multilist(x)
    
# Duplicates --------------------------------------------------
if filter_set.loc["duplicates_OK", "value"] == "False":
    records = pd.read_sql("SELECT decimalLatitude, decimalLongitude, eventDate FROM occurrence_records;", con=conn)
    if len(records[records.duplicated() == True]) >= 1:
        print("!!! Failed test for duplicates")
        
# Test spatial parameters -------------------------------------
records2 = (pd.read_sql("""SELECT * FROM occurrence_records;""", con=conn)
            .astype({'decimalLongitude': 'float',
                     'decimalLatitude': 'float',
                     'radius_meters': 'float'}))

# Make a point geometry from coordinates
gdf = gpd.GeoDataFrame(records2,
                       geometry=gpd.points_from_xy(records2['decimalLongitude'],
                                                   records2['decimalLatitude']))

# Set the coordinate reference system
gdf.crs={'init' :'epsg:4326'}

# Test species extent of occurrence polygon
if filter_set.loc["use_taxon_geometry", "value"] == "True":
    if taxon_concept.loc["TAXON_EOO", "value"] != "None":
        poly = shapely.wkt.loads(taxon_concept.loc["TAXON_EOO", "value"])
        gdf_eoo = gdf[~gdf["geometry"].within(poly)]
        if len(gdf_eoo) >= 1:
            print("!!! Failed species extent of occurrence test.")

# Test query polygon
if filter_set.loc["query_polygon", "value"] != "None":
    poly = shapely.wkt.loads(filter_set.loc["query_polygon", "value"])
    gdf_query = gdf[~gdf["geometry"].within(poly)]
    if len(gdf_query) >= 1:
        print("!!! Failed query polygon test.")



                                     value
key                                       
ID                              TestCuckoo
GBIF_ID                            2496287
EBIRD_ID              Yellow-billed Cuckoo
detection_distance_m                   200
TAXON_EOO                             None
                                 value
parameter                             
name                        test_api_1
query_polygon                     None
issues_omit                       None
sampling_protocols_omit           None
bases_omit                        None
has_coordinate_uncertainty       False
geoissue                          None
default_coordUncertainty          1000
max_coordinate_uncertainty       10000
datasets_omit                     None
collection_codes_omit             None
institutions_omit                 None
use_taxon_geometry               False
lat_range                        30,35
lon_range                      -89,-75
country                             

In [9]:
""" THIS COULD BE USED FOR ACCOUNTING FOR COORD PRECISION}
records = pd.read_sql("SELECT decimalLatitude, decimalLongitude, eventDate FROM occurrence_records;", con=conn)
records["lat_precision"] = [len(x.split(".")[1]) for x in records["decimalLatitude"]]
records["lon_precision"] = [len(x.split(".")[1]) for x in records["decimalLongitude"]]
print(records.head())
print(records["lat_precision"].unique())
print(records["lon_precision"].unique())

pairs = records.filter(['lat_precision', 'lon_precision'], axis=1).drop_duplicates()
print(pairs)
for p in pairs.index:
    print(pairs.iloc[p])"""

' THIS COULD BE USED FOR ACCOUNTING FOR COORD PRECISION}\nrecords = pd.read_sql("SELECT decimalLatitude, decimalLongitude, eventDate FROM occurrence_records;", con=conn)\nrecords["lat_precision"] = [len(x.split(".")[1]) for x in records["decimalLatitude"]]\nrecords["lon_precision"] = [len(x.split(".")[1]) for x in records["decimalLongitude"]]\nprint(records.head())\nprint(records["lat_precision"].unique())\nprint(records["lon_precision"].unique())\n\npairs = records.filter([\'lat_precision\', \'lon_precision\'], axis=1).drop_duplicates()\nprint(pairs)\nfor p in pairs.index:\n    print(pairs.iloc[p])'

In [ ]:
conn.close()